In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from RMtools_1D import calculate_RMSF, rmtools_bwpredict
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
df = pd.read_csv("1603319074-J0419_54-corr.20220927.cube.statistics.tab", sep="\t")
df

In [ ]:
plt.step(
    df["frequency [MHz]"],
    df["rmsStokesV [uJy/beam]"],
)
plt.axhline(np.nanmedian(df["rmsStokesV [uJy/beam]"]), color="k", ls="--")
plt.yscale("log")

In [ ]:
cut = np.nanmedian(df["rmsStokesV [uJy/beam]"])
good_freqs = []
# Find intervals where rmsStokesV is above the median
interval = []
for i, point in enumerate(df["rmsStokesV [uJy/beam]"]):
    if point > cut and df["rmsStokesV [uJy/beam]"][i+1] < cut:
        start = df["frequency [MHz]"][i] * 1e6
        # print("Start of interval: ", start)
        # plt.axvline(start, color="red")
        interval.append(start)
    if point < cut and df["rmsStokesV [uJy/beam]"][i+1] > cut:
        stop = df["frequency [MHz]"][i] * 1e6
        # print("End of interval: ", stop)
        # plt.axvline(stop, color="red")
        interval.append(stop)
    
    if len(interval) == 2:
        print("Interval: ", interval)
        good_freqs.append(interval)
        interval = []

In [ ]:
freq_list = np.loadtxt("freq.txt")
nchan = 1024
for nchan in (
    1024,
    256,
    128,
    64,
    32,
):  
    print(f"""
########################################################################
# nchan = {nchan}
########################################################################
    """
    )
    freqs = np.linspace(freq_list[0], freq_list[-1], nchan)
    idxs = []
    for interval in good_freqs:
        idx = np.where((freqs > interval[0]) & (freqs < interval[1]))
        idxs.extend(idx[0])
        # freqs = np.delete(freqs, idx)
    calculate_RMSF.determine_RMSF_parameters(
        freqs[idxs],
        weights_array=None,
        phi_max=None,
        dphi=None,
    )
    adjoint_info = rmtools_bwpredict.bwdepol_compute_predictions(
        freqs[idxs],
    )
    rmtools_bwpredict.plot_adjoint_info(adjoint_info)
    plt.show()
    print("""
########################################################################
    """)